In [9]:
import numpy as np
import pandas as pd
from requests import get

from datetime import timedelta
from time import sleep, time
import plotly.graph_objects as go
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
params = {
    "symbol": "BTCUSDT",
    "interval": "5m",
    "startTime": None,
    "endTime": None,
    "limit": 10,
}
b_cans = get(url="https://fapi.binance.com/fapi/v1/klines", params=params).json()

In [20]:
pd.DataFrame(b_cans)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1699810500000,37113.40,37136.70,37113.30,37128.60,147.204,1699810799999,5465072.43420,2758,108.136,4014667.69650,0
1,1699810800000,37128.70,37131.30,37124.20,37124.30,65.208,1699811099999,2421123.38830,1639,26.480,983185.89960,0
2,1699811100000,37124.20,37140,37124,37124,205.932,1699811399999,7646242.44770,3153,107.821,4003367.84080,0
3,1699811400000,37124,37130.10,37119,37119,141.271,1699811699999,5244702.88560,2105,52.246,1939658.98040,0
4,1699811700000,37119,37119.10,37093.10,37108.40,236.919,1699811999999,8790678.85290,4223,82.132,3047490.80300,0
5,1699812000000,37108.40,37114.70,37095.40,37100.10,189.440,1699812299999,7029095.49380,3204,93.379,3464794.65150,0
6,1699812300000,37100.10,37100.20,37080.10,37084.60,259.812,1699812599999,9636204.27430,3614,79.260,2939537.07930,0
7,1699812600000,37084.60,37096,37084.60,37090.70,218.572,1699812899999,8106781.97940,2508,149.109,5530359.15690,0
8,1699812900000,37090.70,37113.90,37086.60,37106.60,268.934,1699813199999,9976707.79880,3438,180.351,6690559.58290,0
9,1699813200000,37106.60,37114,37102.30,37113.80,75.147,1699813499999,2788571.21980,1181,54.264,2013663.47090,0


In [14]:
BINANCE_TIMEFRAMES = ["1m", "3m", "5m"]
TIMEFRAMES_IN_MINUTES = [1, 3, 5]


def get_timeframe_in_ms(timeframe: str):
    return int(timedelta(minutes=TIMEFRAMES_IN_MINUTES[BINANCE_TIMEFRAMES.index(timeframe)]).seconds * 1000)


def get_busdm_candles(
    symbol: str,
    timeframe: str,
    since_date_ms: int = None,
    until_date_ms: int = None,
    candles_to_dl: int = 1500,
    limit: int = 1500,
):
    timeframe_in_ms = get_timeframe_in_ms(timeframe=timeframe)
    candles_to_dl_ms = candles_to_dl * timeframe_in_ms

    if until_date_ms is None:
        if since_date_ms is None:
            until_date_ms = int(time() * 1000) - timeframe_in_ms
            since_date_ms = until_date_ms - candles_to_dl_ms
        else:
            until_date_ms = since_date_ms + candles_to_dl_ms - 5000
    else:
        if since_date_ms is None:
            since_date_ms = until_date_ms - candles_to_dl_ms
            until_date_ms -= 5000

    b_candles = []
    params = {
        "symbol": symbol,
        "interval": timeframe,
        "startTime": since_date_ms,
        "endTime": until_date_ms,
        "limit": limit,
    }

    while params["startTime"] + timeframe_in_ms < until_date_ms:
        try:
            b_data = get(url="https://fapi.binance.com/fapi/v1/klines", params=params).json()
            last_candle_time_ms = b_data[-1][0]
            if last_candle_time_ms == params["startTime"]:
                sleep(0.3)
            else:
                b_candles.extend(b_data)
                params["startTime"] = last_candle_time_ms + 2000
        except Exception as e:
            raise Exception(f"get_busdm_candles -> {e}")
    candles_np = np.array(b_candles, dtype=np.float_)[:, :5]
    return candles_np

In [15]:
def busdm_to_df(candles: np.array):
    candles_df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close'])
    candles_df['timestamp'] = candles_df['timestamp'].astype(dtype=np.int64)
    candles_df.set_index(pd.to_datetime(candles_df['timestamp'], unit='ms'), inplace=True)
    candles_df.index.rename('datetime', inplace=True)
    return candles_df

In [16]:
def plot_busdm_candles(candles: np.array):
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=pd.to_datetime(candles[:, 0], unit="ms"),
                open=candles[:, 1],
                high=candles[:, 2],
                low=candles[:, 3],
                close=candles[:, 4],
            )
        ]
    )
    fig.update_layout(height=600, xaxis_rangeslider_visible=False)
    fig.show()

In [17]:
candles_busdm = get_busdm_candles(symbol='BTCUSDT', timeframe='5m', candles_to_dl=10)
candles_busdm

array([[1699809900000.00, 37140.10, 37140.10, 37123.80, 37134.70],
       [1699810200000.00, 37134.60, 37141.90, 37102.60, 37113.30],
       [1699810500000.00, 37113.40, 37136.70, 37113.30, 37128.60],
       [1699810800000.00, 37128.70, 37131.30, 37124.20, 37124.30],
       [1699811100000.00, 37124.20, 37140.00, 37124.00, 37124.00],
       [1699811400000.00, 37124.00, 37130.10, 37119.00, 37119.00],
       [1699811700000.00, 37119.00, 37119.10, 37093.10, 37108.40],
       [1699812000000.00, 37108.40, 37114.70, 37095.40, 37100.10],
       [1699812300000.00, 37100.10, 37100.20, 37080.10, 37084.60],
       [1699812600000.00, 37084.60, 37096.00, 37084.60, 37090.70]])

In [7]:
candles_busdm_df = busdm_to_df(candles=candles_busdm)
candles_busdm_df

,timestamp,open,high,low,close
datetime,,,,,
2023-11-12 17:10:00,1699809000000,37124.1,37139.4,37124.1,37139.4
2023-11-12 17:15:00,1699809300000,37139.3,37140.2,37135.5,37138.7
2023-11-12 17:20:00,1699809600000,37138.7,37142.1,37134.0,37140.0
2023-11-12 17:25:00,1699809900000,37140.1,37140.1,37123.8,37134.7
2023-11-12 17:30:00,1699810200000,37134.6,37141.9,37102.6,37113.3
2023-11-12 17:35:00,1699810500000,37113.4,37136.7,37113.3,37128.6
2023-11-12 17:40:00,1699810800000,37128.7,37131.3,37124.2,37124.3
2023-11-12 17:45:00,1699811100000,37124.2,37140.0,37124.0,37124.0
2023-11-12 17:50:00,1699811400000,37124.0,37130.1,37119.0,37119.0


In [8]:
plot_busdm_candles(candles=candles_busdm)